In [3]:
%matplotlib qt

import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
import math

OMEGA_P = 1

X = 0
Y = 1
dt = 0.00001
t = 0
g = np.array((0.0, -10.0)) * dt * dt
colors = ("black", "brown", "orange", "olive")

number_of_boxes = 4
pos_boxes = np.array(((10.0, 0.01),(11.1, 0.01),(10.5, 1.1),(10.7, 2.2)))
v_boxes = np.array(((0.0, 0.0),(0.0, 0.0),(0.0,0.0),(0.0,0.0)))
h_boxes = 1.0
w_boxes = 1.0
m_boxes = np.array ((1.0, 1.1, 0.9, 1.2))
rot_boxes = 0.0

pos_bird = (2.0, 2.0)
v_bird = (7.0, 2.0)

number_of_pigs = 2
pos_obj = np.array(((8.6, 0.4), (11.0, 1.7),pos_bird))
v_obj = np.array(((0.0, 0.0),(0.0, 0.0),v_bird))
r_obj = np.array((0.4, 0.4, 0.4, 0))
m_obj = np.array ((0.8, 1.3, 2.0, 1.0, 1.0))

BIRD = number_of_pigs
v_obj *= dt

fig, ax = plt.subplots()
plt.gca().set_aspect('equal', adjustable='box') 


def collision_circle_to_circle(pos_1, pos_2, r_1, r_2):
    pos = pos_1 - pos_2
    pos[X] *= pos[X]
    pos[Y] *= pos[Y]
    return r_1 + r_2 >= math.sqrt(pos[X] + pos[Y])

def collision_circle_to_box(pos_c, pos_b, r_c, h_b, w_b):
    if(math.fabs( pos_c[X] - (pos_b[X] + w_b) ) <= r_c) and (pos_c[Y] >= pos_b[Y] - r_c) and (pos_c[Y] <= pos_b[Y] + h_b + r_c):
        return 1
    if(math.fabs( pos_c[X] - pos_b[X] ) <= r_c) and (pos_c[Y] >= pos_b[Y] - r_c) and (pos_c[Y] <= pos_b[Y] + h_b + r_c):
        return 1
    
    if((math.fabs( pos_c[Y] - (pos_b[Y] + h_b) ) <= r_c) and (pos_c[X] >= pos_b[X] - r_c) and (pos_c[X] <= pos_b[X] + w_b + r_c)):
        return 2
    if(math.fabs( pos_c[Y] - pos_b[Y] ) <= r_c) and (pos_c[X] >= pos_b[X] - r_c) and (pos_c[X] <= pos_b[X] + w_b + r_c):
        return 2
    
    return 0

def collision_box_to_box(pos_1, pos_2, h, w):
    if((math.fabs(pos_1[X] - pos_2[X]) <= w) and (math.fabs(pos_1[Y] - pos_2[Y]) <= h)):
        return True
    return False

def collision_circle_to_wall(pos, v, r):    
    if pos[X] - r <= 0.0 and v[X] < 0: 
        return 1
    if pos[X] + r >= 20.0 and v[X] > 0: 
        return 3
    if pos[Y] - r <= 0.0 and v[Y]  < 0:
        return 2
    if pos[Y] + r >= 10.0 and v[Y] > 0:
        return 4
    return  0

def collision_box_to_wall(pos_b, v, h, w):
    if pos_b[X] <= 0.0 and v[X] < 0: 
        return 1
    if pos_b[X] + w >= 20.0 and v[X] > 0: 
        return 3
    if pos_b[Y] <= 0.0 and v[Y] < 0:
        return 2
    if pos_b[Y] + h >= 10.0 and v[Y] > 0:
        return 4
    return  0

def check_impulse(v, m):
    return m * math.sqrt(v[X] * v[X]  + v[Y] * v[Y]) >= OMEGA_P * dt * dt

def collision(pos1, pos2, v1, v2, m1, m2):
    x1 = 1.1
    y1 = 0.0
    pos = pos1 - pos2
    if (math.sqrt(x1*x1 + y1*y1) * math.sqrt(pos[X]*pos[X] + pos[Y]*pos[Y]) != 0 ):
        cos = (x1 * pos[X] + y1 * pos[Y]) / (math.sqrt(x1*x1 + y1*y1) * math.sqrt(pos[X]*pos[X] + pos[Y]*pos[Y]))
    else:
        cos = 1
    sin = math.sqrt(1 - cos * cos)
    s = np.array (((cos, -sin), (sin, cos)))
    speed1 = np.array((0.0, 0.0))
    speed2 = np.array((0.0, 0.0))
    speed1[X] = s[X][X] * v1[X] + s[X][Y] * v1[Y]
    speed1[Y] = s[Y][X] * v1[X] + s[Y][Y] * v1[Y]
    speed2[X] = s[X][X] * v2[X] + s[X][Y] * v2[Y] 
    speed2[Y] = s[Y][X] * v2[X] + s[Y][Y] * v2[Y]            
    u1 = ((m1 - m2) * speed1 + 2 * speed2 * m2) / (m1 + m2)
    u2 = ((m2 - m1) * speed2 + 2 * speed1 * m1) / (m2 + m1)
    s = np.array (((cos, sin), (-sin, cos)))
    u1[X] = s[X][X] * u1[X] + s[X][Y] * u1[Y] 
    u1[Y] = s[Y][X] * u1[X] + s[Y][Y] * u1[Y]
    u2[X] = s[X][X] * u2[X] + s[X][Y] * u2[Y] 
    u2[Y] = s[Y][X] * u2[X] + s[Y][Y] * u2[Y] 
    return u1,u2
                
    
def time_step(n):
    global t, pos_obj, v_obj, BIRD, number_of_pigs, m_obj
    bounce = np.array(((1, 1), (-1,1), (1,-1), (-1,1), (1,-1)))
    bounce_box = np.array(((1,1), (-1,1), (1,-1)))
    for _ in range(n):
        t += dt
            
        for PIG in range(number_of_pigs + 1):
            v_obj[PIG] *= bounce[collision_circle_to_wall(pos_obj[PIG] + v_obj[PIG], v_obj[PIG], r_obj[PIG])]
            
        for BOX in range(number_of_boxes):
            v_boxes[BOX] *= bounce[collision_box_to_wall(pos_boxes[BOX] + v_boxes[BOX], v_boxes[BOX], h_boxes, w_boxes)]
             
             
        for i in range(number_of_pigs + 1):
            for j in range (number_of_boxes):
                kind = collision_circle_to_box(pos_obj[i] + v_obj[i], pos_boxes[j] + v_boxes[j], r_obj[i], h_boxes, w_boxes)
                if(kind > 0):
                    v_obj[i], v_boxes[j] = collision (pos_obj[i], pos_boxes[j] + np.array((h_boxes/2, w_boxes/2)), v_obj[i], v_boxes[j], m_obj[i], m_boxes[j])
        
        
        for i in range(number_of_boxes - 1):
            for j in [x + i + 1 for x in range (number_of_boxes - i - 1)]:
                if(collision_box_to_box(pos_boxes[i] + v_boxes[i], pos_boxes[j] + v_boxes[j], h_boxes, w_boxes)):
                    v_boxes[i], v_boxes[j] = collision (pos_boxes[i] + np.array((h_boxes/2, w_boxes/2)) , pos_boxes[j] + np.array((h_boxes/2, w_boxes/2)), v_boxes[i], v_boxes[j], m_boxes[i], m_boxes[j])
               
        for i in range(number_of_pigs):
            for j in [x + i + 1 for x in range (number_of_pigs - i)]:
                if(collision_circle_to_circle (pos_obj[i] + v_obj[i], pos_obj[j] + v_obj[j], r_obj[i], r_obj[j])):
                    is_first_destroyed = check_impulse(v_obj[j], m_obj[j])
                    is_second_destroyed = check_impulse(v_obj[i], m_obj[i])
                
                    v_obj[i], v_obj[j] = collision (pos_obj[i], pos_obj[j], v_obj[i], v_obj[j], m_obj[i], m_obj[j])
                
                    if (is_first_destroyed) and (j != BIRD) :
                        v_obj[j] = np.array((0.0,0.0))
                        pos_obj[j] = np.array((200.0,200.0))
                        #m_obj = np.delete(m_obj, j)
                    if(is_second_destroyed) and (i != BIRD) : 
                        v_obj[i] = np.array((0.0,0.0))
                        pos_obj[i] = np.array((200.0,200.0))
                        #m_obj = np.delete(m_obj, i)
                        
        for BOX in range(number_of_boxes):
            pos_boxes[BOX] += v_boxes[BOX]
            if(v_boxes[BOX][X] != 0 and v_boxes[BOX][Y] != 0):
                v_boxes[BOX] += g
        
        pos_obj[BIRD] += v_obj[BIRD]
        v_obj[BIRD] += g
        for PIG in range(number_of_pigs):
            #if( (v_objects[PIG] != np.array((0.0, 0.0))).all() ):
            pos_obj[PIG] += v_obj[PIG]
            if(v_obj[PIG][X] != 0 and v_obj[PIG][Y] != 0):
                v_obj[PIG] += g
                           
def draw():
    plt.cla()
    ax.set_xlim(0,20)
    ax.set_ylim(0,10)
    for i in range(number_of_boxes) :
        box = plt.Rectangle(pos_boxes[i], h_boxes, w_boxes, facecolor = colors[i])
        ax.add_artist(box)
        
    for PIG in range(number_of_pigs) :
        pig = plt.Circle(pos_obj[PIG], r_obj[PIG], facecolor = "green")
        ax.add_artist(pig)
        
    bird = plt.Circle(pos_obj[BIRD], r_obj[BIRD], facecolor = "red")
    ax.add_artist(bird)

def anim(n): 
    draw()
    time_step(1000)
    return fig,ax   

anim = animation.FuncAnimation(fig, anim, frames = 200000, interval = 1)
anim